In [ ]:
pip install torch transformers

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
import pandas as pd
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
# Load your dataset (CSV)
df = pd.read_excel('Shuffled_Dataset.xlsx')
df.head()

,Sentence,Emotion
0,ಜನರು ಇತರರ ಕಡೆಗೆ ಸಹಾನುಭೂತಿಯನ್ನು ಪ್ರದರ್ಶಿಸುವುದನ...,Joy
1,ನಗರವು ಭಾರತದಲ್ಲಿನ ಪ್ರಮುಖ ಸಾರಿಗೆ ಕೇಂದ್ರಗಳಲ್ಲಿ ಒಂ...,Neutral
2,ನಾನು ತುಂಬಾ ಚೆನ್ನಾಗಿ ನಿದ್ದೆ ಮಾಡುತ್ತಿದ್ದೆ ಮತ್ತು ...,Joy
3,ಆ ಮಾರುಕಟ್ಟೆಯಲ್ಲಿ ಮಾರುತ್ತಿದ್ದ ಅಸಹ್ಯ ಮಾವುಗಳು ಎಲ್...,Disgust
4,ನಾವು ನಿಮ್ಮ ನೆರವಿನಿಂದ ಸಮಸ್ಯೆಗಳನ್ನು ಪರಿಹರಿಸಬಲ್ಲೆವು,Neutral


In [ ]:
# Initialize the tokenizer for 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Encode emotion labels into numerical format
label_encoder = LabelEncoder()
df['Emotion'] = label_encoder.fit_transform(df['Emotion'])
df.head()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


,Sentence,Emotion
0,ಜನರು ಇತರರ ಕಡೆಗೆ ಸಹಾನುಭೂತಿಯನ್ನು ಪ್ರದರ್ಶಿಸುವುದನ...,3
1,ನಗರವು ಭಾರತದಲ್ಲಿನ ಪ್ರಮುಖ ಸಾರಿಗೆ ಕೇಂದ್ರಗಳಲ್ಲಿ ಒಂ...,4
2,ನಾನು ತುಂಬಾ ಚೆನ್ನಾಗಿ ನಿದ್ದೆ ಮಾಡುತ್ತಿದ್ದೆ ಮತ್ತು ...,3
3,ಆ ಮಾರುಕಟ್ಟೆಯಲ್ಲಿ ಮಾರುತ್ತಿದ್ದ ಅಸಹ್ಯ ಮಾವುಗಳು ಎಲ್...,1
4,ನಾವು ನಿಮ್ಮ ನೆರವಿನಿಂದ ಸಮಸ್ಯೆಗಳನ್ನು ಪರಿಹರಿಸಬಲ್ಲೆವು,4


In [ ]:
import joblib

# Save the fitted label encoder to a file
joblib.dump(label_encoder, 'label_encoder.pkl')

['label_encoder.pkl']

In [ ]:
stop_words = set([
    'ನಾನು', 'ಅದು', 'ಅವರು', 'ಮತ್ತು', 'ಈ', 'ಇದು', 'ಎಂದು', 'ಆ', 'ಅದೇ', 'ಇದನ್ನು',
    'ನಾವು', 'ಅದನ್ನು', 'ನಿನ್ನ', 'ನನಗೆ', 'ಅವನು', 'ಅವಳು', 'ನಿಮ್ಮ', 'ಅವಳ', 'ಅವನ',
    'ನನ್ನ',  'ಮತ್ತು','ಅವರು','ಇಲ್ಲ','ನೀನು','ಹಾಗೂ','ಅವು','ಹಾಗೆ','ಅಲ್ಲಿ','ಇಲ್ಲಿ','ಇವು'
])

word_weight_mapping = {
    'ಸಂತೋಷ': {'label': 'joy', 'weight': 2.0},
    'ಹಾಸ್ಯ': {'label': 'joy', 'weight': 2.0},
    'ಖುಷಿ': {'label': 'joy', 'weight': 2.0},
    'ಉಲ್ಲಾಸ': {'label': 'joy', 'weight': 2.0},
    'ನಗು': {'label': 'joy', 'weight': 2.0},
    'ಭಯ': {'label': 'fear', 'weight': 2.0},
    'ಅಂಜಿಕೆ': {'label': 'fear', 'weight': 2.0},
    'ಹೆದರಿಕೆ': {'label': 'fear', 'weight': 2.0},
    'ಕ್ರೋಧ': {'label': 'anger', 'weight': 2.0},
    'ಸಿಟ್ಟು': {'label': 'anger', 'weight': 2.0},
    'ಕೋಪ': {'label': 'anger', 'weight': 2.0},
    'ದುಃಖ': {'label': 'sadness', 'weight': 2.0},
    'ಬೇಸರ': {'label': 'sadness', 'weight': 2.0},
    'ನಿರಾಸೆ': {'label': 'sadness', 'weight': 2.0},
    'ಆಶ್ಚರ್ಯ': {'label': 'surprise', 'weight': 2.0},
    'ಅಚ್ಚರಿ': {'label': 'surprise', 'weight': 2.0},
    'ವಿಸ್ಮಯ': {'label': 'surprise', 'weight': 2.0},
    'ಆಘಾತ': {'label': 'surprise', 'weight': 2.0},
    'ಬೆರಗು': {'label': 'surprise', 'weight': 2.0},
    'ಅಸಹ್ಯ': {'label': 'disgust', 'weight': 2.0},
    'ದ್ವೇಷ': {'label': 'disgust', 'weight': 2.0},
    'ದುರಾಸೆ': {'label': 'disgust', 'weight': 2.0},
    'ತಿರಸ್ಕಾರ': {'label': 'disgust', 'weight': 2.0}
}


In [ ]:
import re

def preprocess_text(text):
    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s\u0C80-\u0CFF]', '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    # text = text.lower()

    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

In [ ]:
class EmotionDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        sentence = self.dataframe.iloc[index]['Sentence']
        emotion = self.dataframe.iloc[index]['Emotion']

        sentence = preprocess_text(sentence)

        # Tokenize the sentence
        encoding = self.tokenizer.encode_plus(
            sentence,
            max_length=self.max_len,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(emotion, dtype=torch.long),
            'index': index  # Return the index to retrieve the sentence during training
        }

In [ ]:
# Hyperparameters
MAX_LEN = 64
BATCH_SIZE = 16
VALIDATION_SPLIT = 0.2  # 20% of the data for validation

# Split the DataFrame into training and validation sets
train_df, val_df = train_test_split(df, test_size=VALIDATION_SPLIT, random_state=42)

# train_df = pd.read_csv('train_data_6.csv')
# val_df = pd.read_csv('val_data_6.csv')

# Create Dataset and DataLoader for training and validation
train_dataset = EmotionDataset(train_df, tokenizer, MAX_LEN)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset = EmotionDataset(val_df, tokenizer, MAX_LEN)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Save train and test datasets
train_df.to_csv('train_data.csv', index=False)
val_df.to_csv('val_data.csv', index=False)

# # To verify saving, load them back (optional)


In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel
import torch.nn.functional as F

class BERT_LSTM_EmotionClassifier(nn.Module):
    def __init__(self, num_classes, hidden_dim=128, lstm_layers=1, bidirectional=True, dropout=0.3, word_weight_mapping=None, label_encoder=None):
        super(BERT_LSTM_EmotionClassifier, self).__init__()

        # Load the pre-trained BERT model
        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')

        # LSTM layer
        self.lstm = nn.LSTM(
            input_size=self.bert.config.hidden_size,
            hidden_size=hidden_dim,
            num_layers=lstm_layers,
            bidirectional=bidirectional,
            batch_first=True,
        )

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

        # Fully connected layer for classification
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, num_classes)

        # Store the word weight mapping and label encoder
        self.word_weight_mapping = word_weight_mapping if word_weight_mapping is not None else {}
        self.label_encoder = label_encoder

    def forward(self, input_ids, attention_mask, sentences=None):
        # Pass inputs through BERT
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = bert_output.last_hidden_state

        # Pass BERT output through the LSTM
        lstm_output, (h_n, c_n) = self.lstm(sequence_output)

        if self.lstm.bidirectional:
            lstm_output = torch.cat((h_n[-2], h_n[-1]), dim=1)
        else:
            lstm_output = h_n[-1]

        # Apply dropout
        lstm_output = self.dropout(lstm_output)

        # Pass through the fully connected layer for classification
        logits = self.fc(lstm_output)

        # If sentences are provided, apply custom weights
        if sentences is not None:
            logits = self.apply_word_weights(logits, sentences)

        return logits

    def apply_word_weights(self, logits, sentences):
        for i, sentence in enumerate(sentences):
            for word, info in self.word_weight_mapping.items():
                if word in sentence:
                    emotion_label = info['label']
                    weight = info['weight']

                    # Map emotion label to its corresponding index
                    emotion_index = self.label_encoder.transform([emotion_label])[0]

                    # Apply the weight to the logits (consider adding rather than multiplying)
                    logits[i, emotion_index] += weight

        return logits

In [ ]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

model = BERT_LSTM_EmotionClassifier(num_classes=7)  # Adjust `num_classes` based on your dataset

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)

# Gradient clipping value
max_grad_norm = 1.0

# Training function
def train_epoch(model, data_loader, loss_fn, optimizer, device):
    model.train()
    total_loss = 0

    for batch in tqdm(data_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # Pass the actual sentences to apply word weights
        sentences = train_df.iloc[batch['index']]['Sentence'].tolist()

        # Forward pass
        outputs = model(input_ids, attention_mask, sentences=sentences)

        # Calculate loss
        loss = loss_fn(outputs, labels)
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()

    avg_loss = total_loss / len(data_loader)
    print(f"Training loss: {avg_loss}")
    return avg_loss

def eval_model(model, data_loader, loss_fn, device):
    model.eval()
    total_loss = 0
    correct_predictions = 0

    with torch.no_grad():
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()

            _, preds = torch.max(outputs, dim=1)
            correct_predictions += torch.sum(preds == labels)

    avg_loss = total_loss / len(data_loader)
    accuracy = correct_predictions.double() / len(data_loader.dataset)

    print(f"Validation loss: {avg_loss}, accuracy: {accuracy}")
    return avg_loss, accuracy

# Training loop
EPOCHS = 10
for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")
    train_loss = train_epoch(model, train_loader, loss_fn, optimizer, device)
    val_loss, val_acc = eval_model(model, val_loader, loss_fn, device)


Epoch 1/10


100%|██████████| 350/350 [02:04<00:00,  2.82it/s]


Training loss: 1.4030145280701773


100%|██████████| 88/88 [00:09<00:00,  8.83it/s]


Validation loss: 0.8983419046483256, accuracy: 0.7064285714285714
Epoch 2/10


...

Validation loss: 0.594132067064162, accuracy: 0.8507142857142858
Epoch 10/10


100%|██████████| 350/350 [02:05<00:00,  2.78it/s]


Training loss: 0.1910987781334136


100%|██████████| 88/88 [00:09<00:00,  8.93it/s]

Validation loss: 0.6861147802238438, accuracy: 0.83


In [ ]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

model = BERT_LSTM_EmotionClassifier(num_classes=7)  # Adjust `num_classes` based on your dataset

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)

# Gradient clipping value
max_grad_norm = 1.0

# Early stopping parameters
patience = 3  # Number of epochs to wait before stopping if no improvement
best_val_loss = float('inf')  # Initialize best validation loss to infinity
early_stop_counter = 0  # Counter for early stopping

# Training function
def train_epoch(model, data_loader, loss_fn, optimizer, device):
    model.train()
    total_loss = 0

    for batch in tqdm(data_loader):
        # Move input data to GPU (if available)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask)

        # Calculate loss
        loss = loss_fn(outputs, labels)
        total_loss += loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()

        # Gradient clipping
        nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()

    avg_loss = total_loss / len(data_loader)
    print(f"Training loss: {avg_loss}")
    return avg_loss

# Optional: Validation function
def eval_model(model, data_loader, loss_fn, device):
    model.eval()
    total_loss = 0
    correct_predictions = 0

    with torch.no_grad():
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()

            _, preds = torch.max(outputs, dim=1)
            correct_predictions += torch.sum(preds == labels)

    avg_loss = total_loss / len(data_loader)
    accuracy = correct_predictions.double() / len(data_loader.dataset)

    print(f"Validation loss: {avg_loss}, accuracy: {accuracy}")
    return avg_loss, accuracy

# Training loop with early stopping
EPOCHS = 5

for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")

    # Train and evaluate
    train_loss = train_epoch(model, train_loader, loss_fn, optimizer, device)
    val_loss, val_acc = eval_model(model, val_loader, loss_fn, device)

    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0  # Reset the counter
        torch.save(model.state_dict(), 'best_model.pth')  # Save best model
        print("Validation loss improved, model saved.")
    else:
        early_stop_counter += 1
        print(f"No improvement in validation loss for {early_stop_counter} epoch(s).")

    # Stop training if validation loss does not improve for `patience` epochs
    if early_stop_counter >= patience:
        print("Early stopping triggered.")
        break



Epoch 1/5


100%|██████████| 350/350 [01:14<00:00,  4.71it/s]


Training loss: 1.3421733042172024


100%|██████████| 88/88 [00:05<00:00, 16.20it/s]


Validation loss: 0.95271166481755, accuracy: 0.6628571428571428
Validation loss improved, model saved.
Epoch 2/5


...

Validation loss: 0.575415815142068, accuracy: 0.8171428571428572
Validation loss improved, model saved.
Epoch 5/5


100%|██████████| 350/350 [01:14<00:00,  4.73it/s]


Training loss: 0.3948907907094274


100%|██████████| 88/88 [00:05<00:00, 16.06it/s]


Validation loss: 0.5281083166429942, accuracy: 0.8371428571428572
Validation loss improved, model saved.


In [ ]:
# Assuming 'model' is your trained model
torch.save(model.state_dict(), 'kannada_emotion_model_best.pth')

In [ ]:
from google.colab import files

files.download('best_model.pth')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Load the test set
test_df = pd.read_excel('Test_set.xlsx')

# Preprocess the sentences
test_df['Processed_Sentence'] = test_df['Sentence'].apply(preprocess_text)

# Encode the emotions using the label encoder
label_encoder = LabelEncoder()
label_encoder.fit(test_df['Emotion'])  # Fit on the actual labels in the test set
test_df['Encoded_Emotion'] = label_encoder.transform(test_df['Emotion'])

In [ ]:
# Create a custom dataset for the test set
class EmotionTestDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        sentence = self.dataframe.iloc[index]['Processed_Sentence']
        emotion = self.dataframe.iloc[index]['Encoded_Emotion']

        encoding = self.tokenizer.encode_plus(
            sentence,
            max_length=self.max_len,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(emotion, dtype=torch.long)
        }

In [ ]:
model.load_state_dict(torch.load('/content/best_model.pth'))

<ipython-input-19-ed4a627f4e77>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/best_model.pth'))


<All keys matched successfully>

In [ ]:
test_dataset = EmotionTestDataset(test_df, tokenizer, MAX_LEN)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Function to evaluate the model and calculate accuracy
def evaluate_model(model, data_loader, device,test_df,label_encoder,output_file):

    model.eval()
    correct_predictions = 0
    total_predictions = 0
    predicted_emotions = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            # Model outputs the logits
            outputs = model(input_ids, attention_mask)
            _, preds = torch.max(outputs, dim=1)

            # Append the predicted labels (emotion indices)
            predicted_emotions.extend(preds.cpu().numpy())

            # Count correct predictions for accuracy
            correct_predictions += torch.sum(preds == labels).item()
            total_predictions += labels.size(0)

    # Accuracy calculation
    accuracy = correct_predictions / total_predictions
    predicted_emotion_labels = label_encoder.inverse_transform(predicted_emotions)
    test_df['Predicted_Emotion'] = predicted_emotion_labels
    test_df.to_excel(output_file, index=False)

    return accuracy

output_file = "predict_emotion.xlsx"
# Calculate accuracy on the test set
accuracy = evaluate_model(model, test_loader, device,test_df,label_encoder,output_file)
print(f'Test Set Accuracy: {accuracy * 100:.2f}%')

Test Set Accuracy: 69.71%
